<a href="https://colab.research.google.com/github/2025-02-FML-team/WV-Team/blob/change_numpyarray/ChangNarray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PATH와 DIR은 여러분이 가진 폴더와 파일 경로에 맞게 바꾸시길 바랍니다.

싱글몰트가 너무 많아서 개수를 줄였고, 라이와 브랜디가 너무 적어서 라이와 브랜디는 완전히 삭제 했습니다. 즉 category는 4개입니다.

train_image, train_label, test_image, test_label을 나누는 방법을 세가지로 바꿨습니다. 세가지로 바꾼이유는 셋다 accuracy가 안나온다는 것을 보여주기 위함입니다.

image, label, train, test set 나누기 방법 1

In [57]:
import os
import re
import glob
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


# CSV 로드 및 정리, 본인 경로에 맞게 변환
CSV_PATH = "/content/whiskies_filtered.csv"
IMAGE_DIR = "/content/crops/crops"
IMAGE_SIZE = (128, 128)
RANDOM_STATE = 42

df = pd.read_csv(CSV_PATH, dtype={"id": str})
df["id"] = df["id"].astype(str).str.strip().str.replace(r"\.0$", "", regex=True)
df["category"] = df["category"].astype(str).str.strip()

# 라이, 브랜디 완전 삭제
df = df[~df["category"].isin(["라이", "브랜디"])].copy()

# 싱글몰트는 CSV 행(=id 기준)에서 600개 삭제
singlemalt_df = df[df["category"] == "싱글몰트"]
if singlemalt_df.shape[0] > 600:
    # 싱글몰트 중 600개를 랜덤하게 제거
    drop_idx = singlemalt_df.sample(600, random_state=RANDOM_STATE).index
    df = df.drop(index=drop_idx).reset_index(drop=True)

print("최종 category 분포:\n", df["category"].value_counts())

# id → category 매핑
id2cat = dict(zip(df["id"], df["category"]))


# 이미지 매칭 (jpg만 사용)
image_files = glob.glob(os.path.join(IMAGE_DIR, "**", "*.jpg"), recursive=True)

paths, labels, unmatched = [], [], []
for f in image_files:
    fname = os.path.basename(f)
    base = os.path.splitext(fname)[0]
    m = re.search(r"\d{6,}", base)  # 파일명에서 숫자(id) 추출
    if not m:
        unmatched.append(fname)
        continue
    id_part = m.group(0)
    if id_part in id2cat:
        paths.append(f)
        labels.append(id2cat[id_part])
    else:
        unmatched.append(fname)

print(f"매칭된 이미지 수: {len(paths)}")
print(f"매칭되지 않은 이미지 수: {len(unmatched)}")

if len(paths) == 0:
    raise RuntimeError("일치하는 이미지가 없습니다. CSV의 id 형식 또는 파일명 규칙을 확인하세요.")


# 이미지 로드
X_list = []
for p in paths:
    with Image.open(p) as im:
        im = im.convert("RGB")
        im = im.resize(IMAGE_SIZE)
        arr = np.asarray(im, dtype=np.uint8)
        X_list.append(arr)
X = np.stack(X_list, axis=0)


# 라벨 인코딩
le = LabelEncoder()
y_int = le.fit_transform(labels)


# train/test 분리
train_images, test_images, train_labels, test_labels = train_test_split(
    X, y_int, test_size=0.2, random_state=RANDOM_STATE, stratify=y_int
)

print("train_images:", train_images.shape)
print("test_images:", test_images.shape)
print("train_labels 분포:", np.bincount(train_labels))
print("class mapping:", dict(zip(le.classes_, range(len(le.classes_)))))

최종 category 분포:
 category
싱글몰트    543
블렌디드    376
기타      286
버번      186
Name: count, dtype: int64
매칭된 이미지 수: 2726
매칭되지 않은 이미지 수: 1490
train_images: (2180, 128, 128, 3)
test_images: (546, 128, 128, 3)
train_labels 분포: [442 288 591 859]
class mapping: {np.str_('기타'): 0, np.str_('버번'): 1, np.str_('블렌디드'): 2, np.str_('싱글몰트'): 3}


image, label, train, test set 나누기 방법 2

In [47]:
import os
import re
import glob
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# CSV 로드 및 정리
CSV_PATH = "/content/whiskies_filtered.csv"
IMAGE_DIR = "/content/crops/crops"
IMAGE_SIZE = (128, 128)
RANDOM_STATE = 42

df = pd.read_csv(CSV_PATH, dtype={"id": str})
df["id"] = df["id"].astype(str).str.strip().str.replace(r"\.0$", "", regex=True)
df["category"] = df["category"].astype(str).str.strip()

#라이, 브랜디 완전 삭제
df = df[~df["category"].isin(["라이", "브랜디"])].copy()

#싱글몰트는 CSV 행(=id 기준)에서 600개 삭제
singlemalt_df = df[df["category"] == "싱글몰트"]
other_df = df[df["category"] != "싱글몰트"]

if singlemalt_df.shape[0] > 600:
    drop_idx = singlemalt_df.sample(600, random_state=RANDOM_STATE).index
    df = df.drop(index=drop_idx).reset_index(drop=True)

print("최종 category 분포:\n", df["category"].value_counts())

id2cat = dict(zip(df["id"], df["category"]))


#이미지 매칭 (jpg만 사용)
image_files = glob.glob(os.path.join(IMAGE_DIR, "**", "*.jpg"), recursive=True)

# id별 이미지 경로 모으기
id_to_files = {}
for f in image_files:
    fname = os.path.basename(f)
    m = re.search(r"(\d+)_\d+", fname)
    if not m:
        continue
    id_part = m.group(1)
    if id_part not in id_to_files:
        id_to_files[id_part] = []
    id_to_files[id_part].append(f)

# 이미지 로드 및 병합 (128x256)
X_list, y_list = [], []
for id_part, files in id_to_files.items():
    if id_part not in id2cat:
        continue  # CSV에 없는 id는 제외

    # 파일 2개만 사용 (1_label, 2_label)
    files = sorted(files)[:2]
    if len(files) < 2:
        continue  # 짝이 안 맞으면 skip

    imgs = []
    for p in files:
        with Image.open(p) as im:
            im = im.convert("RGB")
            im = im.resize(IMAGE_SIZE)
            imgs.append(np.asarray(im, dtype=np.uint8))

    # 두 이미지를 좌우로 붙이기 (128, 128, 3) + (128, 128, 3) → (128, 256, 3)
    merged = np.concatenate(imgs, axis=1)
    X_list.append(merged)
    y_list.append(id2cat[id_part])

X = np.stack(X_list, axis=0)
labels = np.array(y_list)

print(f"최종 병합된 이미지 수: {len(X_list)}")
print(f"이미지 shape 예시: {X.shape}")

# 라벨 인코딩
le = LabelEncoder()
y_int = le.fit_transform(labels)

# train/test 분리
train_images, test_images, train_labels, test_labels = train_test_split(
    X, y_int, test_size=0.2, random_state=RANDOM_STATE, stratify=y_int
)

print("train_images:", train_images.shape)
print("test_images:", test_images.shape)
print("train_labels 분포:", np.bincount(train_labels))
print("class mapping:", dict(zip(le.classes_, range(len(le.classes_)))))

최종 category 분포:
 category
싱글몰트    543
블렌디드    376
기타      286
버번      186
Name: count, dtype: int64
최종 병합된 이미지 수: 1335
이미지 shape 예시: (1335, 128, 256, 3)
train_images: (1068, 128, 256, 3)
test_images: (267, 128, 256, 3)
train_labels 분포: [214 139 290 425]
class mapping: {np.str_('기타'): 0, np.str_('버번'): 1, np.str_('블렌디드'): 2, np.str_('싱글몰트'): 3}


image, label, train, test set 나누기 방법 3

In [38]:
import os
import re
import glob
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


# CSV 로드 및 정리, 본인 경로에 맞게 변환
CSV_PATH = "/content/whiskies_filtered.csv"
IMAGE_DIR = "/content/crops/crops"
IMAGE_SIZE = (128, 128)
RANDOM_STATE = 42

df = pd.read_csv(CSV_PATH, dtype={"id": str})
df["id"] = df["id"].astype(str).str.strip().str.replace(r"\.0$", "", regex=True)
df["category"] = df["category"].astype(str).str.strip()

#라이, 브랜디 완전 삭제
df = df[~df["category"].isin(["라이", "브랜디"])].copy()

#싱글몰트는 CSV 행(=id 기준)에서 600개 삭제
singlemalt_df = df[df["category"] == "싱글몰트"]
other_df = df[df["category"] != "싱글몰트"]

if singlemalt_df.shape[0] > 600:
    drop_idx = singlemalt_df.sample(600, random_state=RANDOM_STATE).index
    df = df.drop(index=drop_idx).reset_index(drop=True)

print("최종 category 분포:\n", df["category"].value_counts())

# id → category 매핑
id2cat = dict(zip(df["id"], df["category"]))


image_files = glob.glob(os.path.join(IMAGE_DIR, "**", "*.jpg"), recursive=True)

paths, labels, unmatched = [], [], []

for f in image_files:
    fname = os.path.basename(f)

    # _1_ 이 포함된 파일은 건너뜀
    if "_1_" in fname:
        continue

    base = os.path.splitext(fname)[0]
    m = re.search(r"\d{6,}", base)
    if not m:
        unmatched.append(fname)
        continue

    id_part = m.group(0)
    if id_part in id2cat:
        paths.append(f)
        labels.append(id2cat[id_part])
    else:
        unmatched.append(fname)

print(f"매칭된 이미지 수: {len(paths)}")
print(f"매칭되지 않은 이미지 수: {len(unmatched)}")

if len(paths) == 0:
    raise RuntimeError("일치하는 이미지가 없슴. CSV의 id 형식 또는 파일명 규칙 확인")


# 이미지 로드 (_2_ 이미지만 사용)
X_list = []
for p in paths:
    with Image.open(p) as im:
        im = im.convert("RGB")
        im = im.resize(IMAGE_SIZE)
        arr = np.asarray(im, dtype=np.uint8)
        X_list.append(arr)
X = np.stack(X_list, axis=0)


# 라벨 인코딩
le = LabelEncoder()
y_int = le.fit_transform(labels)


# train/test 분리
train_images, test_images, train_labels, test_labels = train_test_split(
    X, y_int, test_size=0.2, random_state=RANDOM_STATE, stratify=y_int
)

print("train_images:", train_images.shape)
print("test_images:", test_images.shape)
print("train_labels 분포:", np.bincount(train_labels))
print("class mapping:", dict(zip(le.classes_, range(len(le.classes_)))))

최종 category 분포:
 category
싱글몰트    543
블렌디드    376
기타      286
버번      186
Name: count, dtype: int64
매칭된 이미지 수: 1391
매칭되지 않은 이미지 수: 754
train_images: (1112, 128, 128, 3)
test_images: (279, 128, 128, 3)
train_labels 분포: [229 149 300 434]
class mapping: {np.str_('기타'): 0, np.str_('버번'): 1, np.str_('블렌디드'): 2, np.str_('싱글몰트'): 3}


In [48]:
import pandas as pd

# train 데이터 분포 확인
train_label_names = le.inverse_transform(train_labels)
test_label_names = le.inverse_transform(test_labels)

print(" Train 라벨")
print(pd.Series(train_label_names).value_counts())

print("\nTest 라벨")
print(pd.Series(test_label_names).value_counts())

 Train 라벨
싱글몰트    425
블렌디드    290
기타      214
버번      139
Name: count, dtype: int64

Test 라벨
싱글몰트    106
블렌디드     73
기타       53
버번       35
Name: count, dtype: int64


여기서부터는 강의록2에 있는 코드입니다.

위에 있는 3가지 방법중에 하나를 고르고 아래 코드를 실행시키면 됩니다.

In [58]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(4, activation="softmax"),
])

In [59]:
model.compile(optimizer= "rmsprop",
              loss = "sparse_categorical_crossentropy",
              metrics=["accuracy"])

방법2를 골랐을 경우 128 * 128 * 3을 128 * 256 * 3으로 바꿔주세요

In [60]:
train_images = train_images.reshape((train_images.shape[0], 128*128*3))
test_images  = test_images.reshape((test_images.shape[0], 128*128*3))
train_images = train_images.astype("float32")/255
test_images = test_images.astype("float32")/255

In [61]:
print(train_images.shape, train_images.dtype)
print(train_labels.shape, train_labels[:10])

(2180, 49152) float32
(2180,) [3 2 2 2 0 3 3 0 2 3]


In [62]:
model.fit(train_images, train_labels, epochs=50, batch_size=128)

Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 12s 561ms/step - accuracy: 0.2839 - loss: 166.0871
Epoch 2/50
14/18 ━━━━━━━━━━━━━━━━━━━━ 2s 581ms/step - accuracy: 0.3005 - loss: 20.7519

KeyboardInterrupt: 

In [32]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.2732 - loss: 7.6455
0.301075279712677
